In [ ]:
import teval.io as tio
import teval.stats as tstats
import teval.viz.static as tviz
import matplotlib.pyplot as plt

In [ ]:
print("Loading Data...")
ds_ensemble = tio.load_ensemble("../data/ensemble_member_*.nc")
gdf_hydro = tio.load_hydrofabric("../data/gage_10023000.gpkg")

In [ ]:
stats = {
        'mean': ds_ensemble.mean(dim="ensemble_member"),
        'median': ds_ensemble.median(dim="ensemble_member"),
        'std': ds_ensemble.std(dim="ensemble_member"),
        'p05': ds_ensemble.quantile(0.05, dim="ensemble_member"), # 5th percentile (Lower bound)
        'p95': ds_ensemble.quantile(0.95, dim="ensemble_member")  # 95th percentile (Upper bound)
    }
    
# Merge and rename
stat_datasets = []
for stat_name, stat_ds in stats.items():
    # Rename variables (e.g., streamflow -> streamflow_mean)
    rename_map = {var: f"{var}_{stat_name}" for var in stat_ds.data_vars}
    stat_datasets.append(stat_ds.rename(rename_map))

In [ ]:
print("Calculating Statistics...")
# This creates one concise file with mean, median, std, p05, p95
ds_summary = tstats.calculate_basics(ds_ensemble)

In [ ]:
print("Plotting Hydrograph...")
fig, ax = plt.subplots(figsize=(12, 6))
tviz.hydrograph(
    ds_summary, 
    feature_id=2860507, 
    var_name="streamflow", 
    ax=ax
)
plt.show()

# Cell 5: Save (The deliverable)
output_file = "../data/gage_10023000_final.nc"
print(f"Saving summary to {output_file}...")
tio.save_ensemble_stats(ds_summary, output_file)